# Demo Cv a .txt. generate_text_files.txt

Este script toma los CV en formato PDF y genera archivos .txt. Se usa la librería **PyMuPdf** https://github.com/pymupdf/PyMuPDF
Además se procesa para eliminar simbolos innecesarios y no reconocidos. Procesa los archivos .pdf de la carpeta **/resumes** y genera como salida .txt en la carpeta **/resumes_text_output**

In [59]:
%run -i '/home/erwin/Genoma/cv-parser/parser/generate_text_files.py'

In [60]:
file = '/home/erwin/Genoma/cv-parser/resumes_text_output/1583796411230-Curriculum__fco_marzo'
cv_txt = open(file, "r")
print(cv_txt.read())

Francisco Javier Contador Ramírez 
Ingeniero Comercial 
C 56 9 9 4624515 
fcocontador@gmail.com 
Milán 12839 Lo Barnechea, Santiago 
Competencias más destacadas: Alta capacidad de trabajo en equipo, manejo de 
trabajo bajo presión, buenas relaciones interpersonales, alta motivación por el aprendizaje, 
gran interés por desarrollar distintas áreas de trabajo, alto grado de responsabilidad en 
distintos sentidos. 
Experiencia Laboral 
Ejecutivo Comercial 
Comercial Portia Mar.2019  Nov 2019 
 Relación comercial con clientes externos en donde manejaba la fuerza de trabajo 
outsourcing. 
 Elaboración de informes internos como externos. 
 Facturación 
 Manejo de equipo de trabajo 
Pension Planning Executive 
Provida Afp Nov.2016  Nov 2018 
 Mantención y Fidelización de cartera de clientes 
 Asesoría Comercial para afiliados y posibles clientes. 
 Elaboración de informes de tendencias de rentabilidad. 
 Proyección de pensiones y productos voluntarios. 
Ejecutivo de Cuentas 
Ecrgroup Empresas

# Demo parser.py
## Puede ser utilizado para filtrar cv para keywords especificas, recuperar mail, nombre, grado más alto, etc.

Primero corremos el script para que procese los archivos ubicados en la carpeta **/resumes**.
Esta funcion genera archivos .JSON

In [61]:
%run -i '/home/erwin/Genoma/cv-parser/parser/parser.py'

[nltk_data] Downloading package punkt to /home/erwin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/erwin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/erwin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Finalizado. Se han procesado 34 CVs


In [62]:
import json
resumes_parser = []
path_to_json = os.getcwd()+'/output_parser/'
cv_parseado = ''
for root, _, filenames in os.walk(os.getcwd()+'/output_parser'):
    for filename in filenames:
        file = os.path.join(root, filename)
        resumes_parser.append(file)

jsons = []
for index, js in enumerate(resumes_parser):
    with open(os.path.join(path_to_json, js)) as json_file:
            jsons.append(json.load(json_file))      

jsons[1]

{'Nombre': 'Kevin Jason Suárez ',
 'Correo': ['kjsuarez@hotmail.com'],
 'Linkedin': None,
 'Celular': '01633323451',
 'Skills': ['Sap', 'Ventas', 'Excel', 'Office'],
 'Grado': ['Ingeniería'],
 'Palabras Claves': ['Análisis',
  'Comprometido',
  'Liderazgo',
  'Estrategia',
  'Negociación',
  'Responsabilidad',
  'Analizar',
  'Gestión'],
 'Educacion': [],
 'Lenguajes': ['Inglés'],
 'Nombre archivo': '/home/erwin/Genoma/cv-parser/1580920711029-Kevin_SQ'}

# Demo seccionar
En principio se seccionan los cv en las siguientes categorias:
1. Extras o Perfil
2. Experiencia
3. Educación
4. Skills

Para poder separar en secciones se ocupa la tabla creada por Daniel, se agregan palabras recurrentes en cada una de las secciones.
Seccionar genera archivos .JSON

In [63]:
import pandas as pd
categorias = pd.read_csv('/home/erwin/Genoma/cv-parser/parser/seccionesCV.csv', header=1)
categorias.head()

,Perfil,Experiencia,Formación Académica,skills
0,Resumen,Profesional,Educación,Habilidades
1,Biografía,Logros,Formación,Cualificaciones
2,Sobre,Experiencia,Estudios,Conocimientos
3,Acerca de mi,historial,Formación educativa,Habilidades
4,personales,trabajo,académico,tecnologías


In [64]:
%run -i '/home/erwin/Genoma/cv-parser/parser/seccionar.py'

Seccionando CVs: 34
Finalizado


In [65]:
import pprint
file_seccionado = open('/home/erwin/Genoma/cv-parser/output_seccionado/1583796411230-Curriculum__fco_marzo.json',)
cv_seccionado = json.load(file_seccionado)
pprint.pprint(cv_seccionado)

{'educación': ': supervisión y gestión punto venta , capacitación , gestión y '
              'pagar \n'
              ' formación académica \n'
              ' universidad américas \n'
              ' titulo ingeniería comercial \n'
              ' universidad américas \n'
              ' título ingeniero ejecución administración empresas , mención '
              'finanzas \n'
              ' saint georges college \n'
              ' colegio amanda labarca \n'
              ' colegio rafael sotomayor \n'
              ' inglés , instituto tronwell 2014 \n'
              ' cursos \n'
              ' curso negociación 2019 \n'
              ' curso tributación lcc chile 2017 \n'
              ' curso decretar ley 3500 lcc chile 2016 \n'
              ' cursos excel asec 2014 \n'
              ' curso liderazgo equipo asec 2014 \n'
              ' ',
 'experiencia': 'ingeniero comercial \n'
                ' c 56 9 9 4624515 \n'
                ' fcocontador@gmail.com \n'
              

# Demo ranking
Para la realización del ranking se usa el curriculum seccionado, por lo tanto se cargan los archivos .JSON

Para el cálculo del ranking se utiliza Tf-idf (frecuencia de término – frecuencia inversa de documento)

\begin{equation*}
idf(t, D) = log \frac{|D|}{|d \in D : t \in D|}
\end{equation*}
Donde D es el número de CV, y el denominador es el número de documentos donde aparece cierta palabra. tf es número de veces que sale una palabra en cierto documento.

\begin{equation*}
\text{Tf-idf} = tf \cdot idf
\end{equation*}

Tf-idf es el producto de dos medidas, frecuencia de término y frecuencia inversa de documento
El factor de frecuencia inversa de documento que atenúa el peso de los términos que ocurren con mucha frecuencia en la colección de documentos e incrementa el peso de los términos que ocurren pocas veces.

## Cálculo de ranking paso a paso:
1. Cargar los archivos .json que contiene a los cvs seccionados
2. Recibir una descripción de cargo.
3. Ampliar la descripción usando cargos parecidos. Usando word embeddings en español (https://github.com/dccuchile/spanish-word-embeddings). Su usa FastText embeddings from SBWC.
3. Recorrer los cvs en sus secciones educación, experiencia y skills. Usando las palabras de la descripción de cargo se calcula para cada uno la métrica Tf-idf.

Antes de pasar al cálculo de ranking veamos unos ejemplos usando word embeddings en español:

In [71]:
from gensim.models.keyedvectors import KeyedVectors


wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/fasttext-sbwc.3.6.e20.vec'
cantidad = 100000
model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)




# definamos algunas funciones utiles

def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals


In [67]:
model.most_similar(positive=['carisma'],topn=4)

[('carismática', 0.7194429636001587),
 ('carismático', 0.7106848955154419),
 ('talento', 0.6735696792602539),
 ('carismáticos', 0.6628715991973877)]

In [68]:
model.similarity('programador', 'desarrollador')

0.7731334

In [72]:
frase_1 = 'software developer'
frase_2 = 'web developer'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.8897964


In [69]:
prc_description = '''ingeniería máster python excel desarrollo  experiencia manejo 
machine learning metodologías ágiles liderar equipos planificar organizar '''







# Expandir descripcion
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
print(word_value.keys())

dict_keys(['ingeniería', 'ingenierías', 'mecatrónica', 'máster', 'másteres', 'licenciatura', 'python', 'perl', 'scripts', 'excel', 'powerpoint', 'word', 'desarrollo', 'sostenible', 'socioeconómico', 'experiencia', 'experiencias', 'pericia', 'manejo', 'manejar', 'manejan', 'machine', 'machines', 'rage', 'learning', 'learn', 'education', 'metodologías', 'metodología', 'metodológicas', 'ágiles', 'ágil', 'veloces', 'liderar', 'encabezar', 'lidera', 'equipos', 'equipo', 'clubes', 'planificar', 'planear', 'planificarse', 'organizar', 'organizarlo'])


In [73]:
# /home/erwin/Genoma/cv-parser/parser/ranking.py

%run -i '/home/erwin/Genoma/cv-parser/parser/ranking.py'

[(39.622961647214886, '1581023751928-CV'),
 (14.96935157846756, '1581000554896-Nabib_M_Garcia_Hirsh_CV'),
 (14.159443311329925, '1588283816605-CV_Erika_Sotomayor_Ulloa'),
 (11.710035350368505, '1581008153040-CV_Jose_Chávez'),
 (11.281740896045001, '1580573732905-DiegoAlbertoGutierrezBejarano'),
 (11.200322658831475, '1581295975312-CV_Renzo_Salvatierra_R'),
 (10.711771880025042, '1581631090164-CV_ROSSANA_ALAYZA_-_FEBRERO_2020'),
 (10.488366254374945, '1580497335052-Jose_Medina_13-1-2020'),
 (9.609904229732464, '1580320654107-CV_Gabriel_Garcia_Lorca'),
 (9.185048017098858, '1580913938357-CV_Gianfranco_Vela_ESP_Ene.2020'),
 (8.930319389160072, 'prueba_genoma_experiencia'),
 (8.611405855192073, '1581690552715-CV_Gustavo_Martinez_Raya'),
 (7.595220569596056, '1581954259745-CV_2020_ANTONIO_HARTMANN_R'),
 (7.200383452766531, '1574824594718-Ing._Comercial_Paula_Fetis_CV'),
 (7.051860557577752, '1583336373286-CV_Gf_Rosales_2020'),
 (6.933161232431816, '1583871756395-CVCabanaGarayChristian1_1'),

In [74]:
file_ra = '/home/erwin/Genoma/cv-parser/resumes_text_output/1581023751928-CV'
cv_txt = open(file_ra, "r")
print(cv_txt.read())

Contactar 
56971920517 Mobile 
chris.hevia@gmail.com 
www.linkedin.com/in/christian 
heviab3902b31 LinkedIn 
Aptitudes principales 
Java 
Gestión de proyectos 
Negociación 
HonorsAwards 
1º Lugar 4to medio 
Mejor de la Carrera Informática 
Análista Programador 
Mejor de la Carrera Informática 
Ingeniería Informática 
Christian Hevia 
Director de Operaciones en Icarus Solutions 
Chile 
Extracto 
Profesional Titulado, con distinción máxima, con 6 años de 
experiencia en Jefatura de Proyectos. 
Capacitado con conocimientos en diversas áreas de la informática, 
principalmente orientado al desarrollo de software, aplicaciones de 
alto rendimiento con énfasis en la calidad del software, liderazgo 
de equipos, alto nivel de análisis y gran motivación para enfrentar 
diversos desafíos. He trabajado en importantes proyectos con 
diversas tecnologías, dentro de las cuales destacan J2ee, Jpa, 
Asp, JavaScript, Xhtml, Metodologías Ágiles, entre otras. 
Sólidos conocimientos en Gestión, Planificaci